In [ ]:
import numpy as np
import cv2 
import os
from glob import glob
import matplotlib.pyplot as plt

from keras.applications import ResNet50
from keras.models import Model

In [ ]:
images_path = 'Flickr_Data/Images/'
images = glob(images_path+'*.jpg')

In [ ]:
captions_path = "Flickr_Data/Flickr_TextData/FLickr8k.token.txt"

In [ ]:
resnet_model = ResNet50(include_top=True)

In [ ]:
# restructure model
resnet_model = Model(inputs=resnet_model.input, outputs=resnet_model.layers[-2].output)

In [ ]:
# preprocess the images
img_feature_vectors = {}

count = 0
for item in images:
    img = cv2.imread(item)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img.reshape(1, 224, 224, 3)
    
    feature_vector = resnet_model.predict(img, verbose=0).reshape(2048,)
    
    img_filename = item.split('\\')[-1]
    img_feature_vectors[img_filename] = feature_vector
    
    count += 1
    
    if (count%50==0):
        print(count)
    
    if (count==1499):
        break

In [ ]:
np.save('image_feature_vectors.npy', img_feature_vectors)

In [ ]:
# load in dictionary of image feature vectors
img_feature_vectors = np.load('image_feature_vectors.npy', allow_pickle='TRUE').item()

In [ ]:
# makes a string lowercase, prepends it with the string 'sos' and appends with 'eos'
def process_string(s):
    s = s.lower()
    return 'sos ' + s + ' eos'

In [ ]:
# preprocess the captions data
captions_dict = {}

f = open(captions_path, 'r').read().split('\n')

# generate a dictionary of filenames to a list of captions
for line in f:
    try:
        filename_caption = line.split('\t') 
        filename = filename_caption[0][:-2]
        caption = filename_caption[1]
        
        if filename in img_feature_vectors:
            if filename not in captions_dict:
                captions_dict[filename] = [caption]
            else:
                captions_dict[filename].append(caption)
    except:
        pass

# prepend and append 'sos' and 'eos' to each caption in the dictionary
for filename, captions in captions_dict.items():
    for i, item in enumerate(captions):
        captions[i] = process_string(item)